In [21]:
import pickle
import numpy as np
import networkx as nx
from scipy import stats
from nilearn import datasets
import json
from graph import cal_graph
from esfMRI import stats_tests
from statsmodels.stats import nonparametric

In [22]:
atlas_labels = datasets.fetch_atlas_aal()["labels"]

In [23]:
with open("datasets.json", "r") as f:
    dataset = json.load(f)

In [5]:
with open("time_series2.pkl", "rb") as f:
    time_series = pickle.load(f)

In [6]:
# 对每个脑区的时间序列进行统计学检验
cnt_change = 0
cnt_nochange = 0
for i in range(116):
    ts_preop = []
    ts_postop = []
    for sub in time_series:
        for run in time_series[sub]["ses-preop"]:
            ts_preop.append(time_series[sub]["ses-preop"][run]["time_series"].T[i])
    for sub in time_series:
        for run in time_series[sub]["ses-postop"]:
            ts_postop.append(time_series[sub]["ses-postop"][run]["time_series"].T[i])
    # time_series["sub-292"]["ses-preop"]["run-02"]["time_series"].T[i]
    # time_series["sub-292"]["ses-postop"]["run-01"]["time_series"].T[i]
    ts_preop = np.hstack(ts_preop)
    ts_postop = np.hstack(ts_postop)

    # Wilcoxon秩和检验
    rs_stat, p_value_rs = stats.ranksums(ts_preop, ts_postop)
    # print("Wilcoxon秩和检验: ", rs_stat, p_value_rs)
    # 曼-惠特尼U检验
    mw_stat, p_value_m = stats.mannwhitneyu(ts_preop, ts_postop)
    # print("曼-惠特尼U检验: ", mw_stat, p_value_m)
    if p_value_rs < 0.01 and p_value_m < 0.01:
        print(atlas_labels[i], "存在明显差异")
        cnt_change += 1
    elif p_value_rs >= 0.01 and p_value_m >= 0.01:
        # print(atlas_labels[i], "不存在明显差异")
        cnt_nochange += 1
    else:
        print(atlas_labels[i], "无法确定")
print(cnt_change, cnt_nochange)

Frontal_Med_Orb_R 存在明显差异
Rectus_L 存在明显差异
Vermis_6 存在明显差异
3 113


In [4]:
with open("FCs.pkl", "rb") as f:
    FCs = pickle.load(f)

In [5]:
FCs["sub-292"]["ses-preop"]["run-01"].shape

(116, 116)

In [12]:
# 对每个脑区的功能连接进行统计学检验
cnt_change = 0
cnt_nochange = 0
p_threshold = 0.001
for i in range(116):
    fc_preop = []
    fc_postop = []
    for sub in FCs:
        for run in FCs[sub]["ses-preop"]:
            fc_preop.append(FCs[sub]["ses-preop"][run][i])
    for sub in FCs:
        for run in FCs[sub]["ses-postop"]:
            fc_postop.append(FCs[sub]["ses-postop"][run][i])
    fc_preop = np.hstack(fc_preop)
    fc_postop = np.hstack(fc_postop)

    # Wilcoxon秩和检验
    rs_stat, rs_p_value = stats.ranksums(fc_preop, fc_postop)
    # print("Wilcoxon秩和检验: ", rs_stat, p_value_rs)
    # 曼-惠特尼U检验
    mw_stat, m_p_value = stats.mannwhitneyu(fc_preop, fc_postop)
    # print("曼-惠特尼U检验: ", mw_stat, p_value_m)
    if rs_p_value < p_threshold and m_p_value < p_threshold:
        if rs_p_value == 0 and m_p_value ==0:
            print(atlas_labels[i], "error")
        # print(atlas_labels[i], "存在明显差异")
        # print(rs_p_value)
        cnt_change += 1
    elif rs_p_value >= p_threshold and m_p_value >= p_threshold:
        # print(atlas_labels[i], "不存在明显差异")
        cnt_nochange += 1
    else:
        print(atlas_labels[i], "无法确定")
print(cnt_change, cnt_nochange)

Vermis_1_2 error
98 18


In [5]:
graphs = {}
threshold = 45
for subid in FCs:
    # if len(FCs[subid]["ses-preop"]) == 0:
    #     continue
    graphs[subid] = {
        "ses-preop":{},
        "ses-postop":{}
    }
    for run, fc in FCs[subid]["ses-preop"].items():
        graphs[subid]["ses-preop"][run] = cal_graph(fc, threshold)
    for run, fc in FCs[subid]["ses-postop"].items():
        graphs[subid]["ses-postop"][run] = cal_graph(fc, threshold)
with open("graph_theory/graphs_static.pkl", "wb") as f:
    pickle.dump(graphs, f)

In [24]:
with open("graph_theory/graphs_static.pkl", "rb") as f:
    graphs = pickle.load(f)

In [25]:
core_num_preop = []
core_num_postop = []

# degree_centrality_preop = []
# degree_centrality_postop = []
# closeness_centrality_preop = []
# closeness_centrality_postop = []
# betweenness_centrality_preop = []
# betweenness_centrality_postop = []
# eigenvector_centrality_preop = []
# eigenvector_centrality_postop = []

# triangles_preop = []
# triangles_postop = []
clustering_preop = []
clustering_postop = []
transitivity_preop = []
transitivity_postop = []

degree_assortativity_coefficient_preop = []
degree_assortativity_coefficient_postop = []
# average_neighbor_degree_preop = []
# average_neighbor_degree_postop = []
# average_degree_connectivity_preop = []
# average_degree_connectivity_postop = []

global_efficiency_preop = []
global_efficiency_postop = []
local_efficiency_preop = []
local_efficiency_postop = []
ratio_preop = []
ratio_postop = []

for subid, graph in graphs.items():
    if len(graph["ses-preop"]) > 0:
        for run in graph["ses-preop"]:
            core_num_preop.append(graph["ses-preop"][run]["k_core"])
            
            # degree_centrality_preop.append(list(graph["ses-preop"][run]["degree_centrality"].values()))
            # closeness_centrality_preop.append(list(graph["ses-preop"][run]["closeness_centrality"].values()))
            # betweenness_centrality_preop.append(list(graph["ses-preop"][run]["betweenness_centrality"].values()))
            # if "eigenvector_centrality" in graph["ses-preop"][run]:
            #     eigenvector_centrality_preop.append(list(graph["ses-preop"][run]["eigenvector_centrality"].values()))
            
            # triangles_preop.append(list(graph["ses-preop"][run]["triangles"].values()))
            clustering_preop.append(list(graph["ses-preop"][run]["clustering"].values()))
            transitivity_preop.append(graph["ses-preop"][run]["transitivity"])
            
            degree_assortativity_coefficient_preop.append(graph["ses-preop"][run]["degree_assortativity_coefficient"])
            # average_neighbor_degree_preop.append(list(graph["ses-preop"][run]["average_neighbor_degree"].values()))
            # average_degree_connectivity_preop.append(graph["ses-preop"][run]["average_degree_connectivity"])
            
            global_efficiency_preop.append(graph["ses-preop"][run]["global_efficiency"])
            local_efficiency_preop.append(graph["ses-preop"][run]["local_efficiency"])
            ratio_preop.append(graph["ses-preop"][run]["ratio"])
    if len(graph["ses-postop"]) > 0:
        for run in graph["ses-postop"]:
            core_num_postop.append(graph["ses-postop"][run]["k_core"])
            
            # degree_centrality_postop.append(list(graph["ses-postop"][run]["degree_centrality"].values()))
            # closeness_centrality_postop.append(list(graph["ses-postop"][run]["closeness_centrality"].values()))
            # betweenness_centrality_postop.append(list(graph["ses-postop"][run]["betweenness_centrality"].values()))
            # if "eigenvector_centrality" in graph["ses-postop"][run]:
            #     eigenvector_centrality_postop.append(list(graph["ses-postop"][run]["eigenvector_centrality"].values()))
            
            # triangles_postop.append(list(graph["ses-postop"][run]["triangles"].values()))
            clustering_postop.append(list(graph["ses-postop"][run]["clustering"].values()))
            transitivity_postop.append(graph["ses-postop"][run]["transitivity"])
            
            degree_assortativity_coefficient_postop.append(graph["ses-postop"][run]["degree_assortativity_coefficient"])
            # average_neighbor_degree_postop.append(list(graph["ses-postop"][run]["average_neighbor_degree"].values()))
            # average_degree_connectivity_postop.append(list(graph["ses-postop"][run]["average_degree_connectivity"].values()))
            
            global_efficiency_postop.append(graph["ses-postop"][run]["global_efficiency"])
            local_efficiency_postop.append(graph["ses-postop"][run]["local_efficiency"])
            ratio_postop.append(graph["ses-postop"][run]["ratio"])

In [26]:
# k core最大值
core_num_preop = np.array(core_num_preop)
core_num_postop = np.array(core_num_postop)

# 中心性，非单个数值，每个节点对应一个
# degree_centrality_preop = np.array(degree_centrality_preop)
# degree_centrality_postop = np.array(degree_centrality_postop)
# closeness_centrality_preop = np.array(closeness_centrality_preop)
# closeness_centrality_postop = np.array(closeness_centrality_postop)
# betweenness_centrality_preop = np.array(betweenness_centrality_preop)
# betweenness_centrality_postop = np.array(betweenness_centrality_postop)
# eigenvector_centrality_preop = np.array(eigenvector_centrality_preop)
# eigenvector_centrality_postop = np.array(eigenvector_centrality_postop)

# 三角形数，与聚类系数相关
# triangles_preop = np.array(triangles_preop)
# triangles_postop = np.array(triangles_postop)
# 聚类系数，每节点对应一个值，可取整体平均值
clustering_preop = np.array(clustering_preop)
clustering_postop = np.array(clustering_postop)
# 转移效率，单个值
transitivity_preop = np.array(transitivity_preop)
transitivity_postop = np.array(transitivity_postop)

# 网络关联性
degree_assortativity_coefficient_preop = np.array(degree_assortativity_coefficient_preop)
degree_assortativity_coefficient_postop = np.array(degree_assortativity_coefficient_postop)
# average_neighbor_degree_preop = np.array(average_neighbor_degree_preop)
# average_neighbor_degree_postop = np.array(average_neighbor_degree_postop)
# average_degree_connectivity_preop = np.array(average_degree_connectivity_preop)
# average_degree_connectivity_postop = np.array(average_degree_connectivity_postop)

# 全局与局部效率
global_efficiency_preop = np.array(global_efficiency_preop)
global_efficiency_postop = np.array(global_efficiency_postop)
local_efficiency_preop = np.array(local_efficiency_preop)
local_efficiency_postop = np.array(local_efficiency_postop)
ratio_preop = np.array(ratio_preop)
ratio_postop = np.array(ratio_postop)

In [8]:
runs = {}
for sub in graphs:
    if len(graphs[sub]["ses-preop"].keys()) == 0:
           print(sub)
    else:
        runs[sub] = [len(graphs[sub]["ses-preop"].keys()), len(graphs[sub]["ses-postop"].keys())]

In [37]:
Cpre =clustering_preop.sum(axis=1)/116
Cpost = clustering_postop.sum(axis=1)/116

In [38]:
stats_tests(Cpre, Cpost)

正态性检验preop:  0.09151604771614075
正态性检验postop:  0.030246388167142868
方差齐性检验:  0.912211021573947
非参数检验: 
Wilcoxon秩和检验:  -2.0039520624506926 0.04507519577097762
曼-惠特尼U检验:  3667.0 0.045216996456419176
ks检验:  0.15254237288135594 0.20726690686401517
kruskal检验 4.015823868600364 0.04507519577097825


array([-1.        , -1.        ,  0.0450752 ,  0.045217  ,  0.20726691,
        0.0450752 ])

In [13]:
lastpre = 0
lastpost = 0
cnt_change = 0
cnt2 = 0
for sub, val in runs.items():
    pre, post = val
    # print(lastpre+pre, lastpost+post)
    if nonparametric.rank_compare_2indep(Cpre[lastpre:lastpre+pre], Cpost[lastpost:lastpost+post])[0] > 0:
        cnt_change += 1
        print(f"{sub}电刺激后聚类系数降低: {dataset[sub]['soz']}")
    else:
        cnt2 += 1
        print(f"{sub}电刺激后聚类系数增加: {dataset[sub]['soz']}")
    lastpre += pre
    lastpost += post
print(cnt_change, cnt2)

sub-292电刺激后聚类系数降低: 左内侧颞叶、左额叶
sub-294电刺激后聚类系数增加: 右前额叶
sub-302电刺激后聚类系数增加: 左内侧颞叶
sub-303电刺激后聚类系数增加: 右内侧颞叶，左内侧颞叶脑软化
sub-307电刺激后聚类系数增加: 左侧脑岛
sub-314电刺激后聚类系数降低: 双侧内侧颞叶
sub-316电刺激后聚类系数增加: 右内侧颞叶
sub-320电刺激后聚类系数增加: 右侧海马体
sub-330电刺激后聚类系数增加: 左枕叶
sub-331电刺激后聚类系数增加: 左内侧颞叶
sub-334电刺激后聚类系数增加: 右颞极，左颞基底
sub-335电刺激后聚类系数降低: 双侧内侧颞叶
sub-352电刺激后聚类系数增加: 左额囊性瘤
sub-372电刺激后聚类系数降低: 左颞极
sub-376电刺激后聚类系数增加: 右内侧颞叶
sub-384电刺激后聚类系数降低: 右内侧颞叶，右额叶
sub-399电刺激后聚类系数增加: 右内侧颞叶，可能是右额基
sub-400电刺激后聚类系数降低: 左内侧颞叶
sub-405电刺激后聚类系数增加: 左额脑软化症
sub-413电刺激后聚类系数降低: 右内侧颞叶
7 13


c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:473: RuntimeWarning: invalid value encountered in double_scalars
  S1 /= nobs1 - 1
c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  wbfn /= (nobs1 + nobs2) * np.sqrt(nobs1 * S1 + nobs2 * S2)
c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:485: RuntimeWarning: invalid value encountered in double_scalars
  df = df_numer / df_denom


In [14]:
nonparametric.rank_compare_2indep(Cpre, Cpost)

<class 'statsmodels.stats.nonparametric.RankCompareResult'>
statistic = -2.027474587551461
pvalue = 0.044351367793845825
s1 = 1175.583063063063
s2 = 437.3360133275387
var1 = 0.08442854517832972
var2 = 0.07774862459156244
var = 0.3444279128648754
var_prob = 0.0017846005847921005
nobs1 = 75
nobs2 = 118
nobs = 193
mean1 = 86.89333333333333
mean2 = 103.42372881355932
prob1 = 0.4143502824858757
prob2 = 0.5856497175141243
somersd1 = -0.17129943502824863
somersd2 = 0.17129943502824863
df = 152.85606948799494
use_t = True
tuple = (-2.027474587551461, 0.044351367793845825)

In [30]:
stats_tests(global_efficiency_preop, global_efficiency_postop)

正态性检验preop:  0.20869816839694977
正态性检验postop:  0.16136741638183594
方差齐性检验:  0.5775167103999672
t检验:  -1.8255820743419626 0.06947495169589021
ANOVA检验:  3.3327499101587237 0.06947495169588921
非参数检验: 
Wilcoxon秩和检验:  -1.5148608598736764 0.12980764416117632
曼-惠特尼U检验:  3852.0 0.13014280458275027
ks检验:  0.1392090395480226 0.2991995414934301
kruskal检验 2.2948034247771147 0.12980764416118593


array([0.06947495, 0.06947495, 0.12980764, 0.1301428 , 0.29919954,
       0.12980764])

In [16]:
stats_tests(local_efficiency_preop, local_efficiency_postop)

正态性检验preop:  0.005230378359556198
正态性检验postop:  0.161850243806839
方差齐性检验:  0.39641565682644664
非参数检验: 
Wilcoxon秩和检验:  -1.8585465697926606 0.06309143654040247
曼-惠特尼U检验:  3722.0 0.06327919203615855
ks检验:  0.14451977401129942 0.26185498589507383
kruskal检验 3.4541953520880497 0.06309143654040299


array([-1.        , -1.        ,  0.06309144,  0.06327919,  0.26185499,
        0.06309144])

In [17]:
stats_tests(ratio_preop, ratio_postop)

正态性检验preop:  0.015562633983790874
正态性检验postop:  0.00028129422571510077
方差齐性检验:  0.26232257718487934
非参数检验: 
Wilcoxon秩和检验:  1.2108311934068827 0.22596010746852468
曼-惠特尼U检验:  4883.0 0.22646723063532503
ks检验:  0.15615819209039547 0.18642232844704254
kruskal检验 1.4661121789271192 0.22596010746852374


array([-1.        , -1.        ,  0.22596011,  0.22646723,  0.18642233,
        0.22596011])

In [18]:
stats_tests(core_num_preop, core_num_postop)

正态性检验preop:  0.011922935023903847
正态性检验postop:  0.00012727353896480054
方差齐性检验:  0.13855632959172676
非参数检验: 
Wilcoxon秩和检验:  -1.5082515192983115 0.13149016609003328
曼-惠特尼U检验:  3854.5 0.13174259784490489
ks检验:  0.15254237288135594 0.20726690686401517
kruskal检验 2.275836961722744 0.13140416980377903


array([-1.        , -1.        ,  0.13149017,  0.1317426 ,  0.20726691,
        0.13140417])

In [19]:
stats_tests(transitivity_preop, transitivity_postop)

正态性检验preop:  0.08317859470844269
正态性检验postop:  0.02004168927669525
方差齐性检验:  0.5523067366554262
非参数检验: 
Wilcoxon秩和检验:  -1.8453278886419304 0.06498985595665809
曼-惠特尼U检验:  3727.0 0.06518226277758184
ks检验:  0.14282485875706213 0.27172657339153816
kruskal检验 3.405235016599704 0.06498985595665759


array([-1.        , -1.        ,  0.06498986,  0.06518226,  0.27172657,
        0.06498986])

In [34]:
stats_tests(degree_assortativity_coefficient_preop, degree_assortativity_coefficient_postop)

正态性检验preop:  0.3088219165802002
正态性检验postop:  0.1359705775976181
方差齐性检验:  0.2952096763872304
t检验:  0.9989193632019863 0.31909759138503113
ANOVA检验:  0.997839894179864 0.31909759138503
非参数检验: 
Wilcoxon秩和检验:  0.9676074602334478 0.33324045003874814
曼-惠特尼U检验:  4791.0 0.33390129357496756
ks检验:  0.156045197740113 0.1869527217840551
kruskal检验 0.9362641970993764 0.33324045003875746


array([0.31909759, 0.31909759, 0.33324045, 0.33390129, 0.18695272,
       0.33324045])